## 20 Newsgroups

In [37]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import numpy as np
from sklearn import datasets, model_selection
import tarfile
import re
import os
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import string
import spacy
from string import digits
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score 
from sklearn.pipeline import Pipeline
from spacy.en import English

Getting category names, i.e. folder names of 20 newsgroups folder

In [2]:
path = "20_newsgroups"
for root, dirs, files in os.walk(path, topdown=False):
    newsgroups = np.array(dirs)
print newsgroups

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


Reading files from first folder. Just use a for loop to read in from more folders.

In [53]:
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(newsgroups)):
    x = []
    y=[]
    path = "20_newsgroups" + "/" + newsgroups[i]
    for root, dirs, files in os.walk(path):
        for name in files:
            f = open(path + "/" + name)
            x.append(f.read())
            y.append(newsgroups[i])
    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x, y, test_size = 0.3, random_state = 1)
    x_train.extend(X_train)
    x_test.extend(X_test)
    y_train.extend(Y_train)
    y_test.extend(Y_test)
#print x[1]

In [4]:
print len(x_train)
print len(x_test)
print len(y_train)
print len(y_test)

13997
6000
13997
6000


In [5]:
y_train = pd.DataFrame(y_train, columns = ['y_train'])
print 'Counts for each category \n%s.' % y_train.y_train.value_counts()

y_test = pd.DataFrame(y_test, columns = ['y_test'])

print 'Counts for each category \n%s.' % y_test.y_test.value_counts()

Counts for each category 
comp.os.ms-windows.misc     700
comp.windows.x              700
rec.motorcycles             700
sci.med                     700
talk.politics.mideast       700
rec.sport.baseball          700
comp.graphics               700
comp.sys.ibm.pc.hardware    700
rec.autos                   700
misc.forsale                700
sci.electronics             700
sci.space                   700
talk.politics.guns          700
talk.politics.misc          700
rec.sport.hockey            700
comp.sys.mac.hardware       700
sci.crypt                   700
talk.religion.misc          700
alt.atheism                 700
soc.religion.christian      697
Name: y_train, dtype: int64.
Counts for each category 
talk.politics.guns          300
sci.space                   300
rec.motorcycles             300
sci.med                     300
talk.politics.misc          300
talk.politics.mideast       300
rec.sport.hockey            300
comp.sys.mac.hardware       300
rec.sport.baseball     

Obtain metadatas From, Subject, Date, Organization, Sender from the headers of each file. Obtain Content from the body of each file and place in the original dataframe.

In [6]:
# Creating the dataframe and assigning columns
def create_dataframe(x): 
    original = pd.DataFrame()
    original = original.assign(Path = '')
    original = original.assign(From = '')
    original = original.assign(Subject = '')
    original = original.assign(Date = '')
    original = original.assign(Organization = '')
    original = original.assign(Sender = '')
    original = original.assign(Content = '')
    original = original.assign(Lines = '')
    original = original.assign(Nntp_posting_host = '')
    original = original.assign(Message_id = '')
    original = original.assign(Reply_to = '')
    original = original.assign(References = '')
    #print original.dtypes

    # Placing the metadatas and content in the respective cells of the dataframe
    for i in range(len(x)):
        data = x[i].split("\n\n", 1)
        email_content = ''
        meta = sorted(data[0].split("\n"))
        for k in meta:
            z = np.array(k.split(":", 1))
            meta_title = z[0].capitalize().replace('-', '_')
            if meta_title in list(original):
                try:
                    original.at[i, meta_title] = z[1]
                except ValueError:
                    original.at[i, z[0]] = np.NaN
        original.at[i, 'Content'] = data[1]
    return original

In [7]:
train_x = create_dataframe(x_train)
train_x.count()

Path                 13997
From                 13997
Subject              13997
Date                 13997
Organization         13415
Sender                7618
Content              13997
Lines                13953
Nntp_posting_host     5974
Message_id           13997
Reply_to              2102
References            8750
dtype: int64

In [8]:
test_x = create_dataframe(x_test)
test_x.count()

Path                 6000
From                 6000
Subject              6000
Date                 6000
Organization         5731
Sender               3254
Content              6000
Lines                5984
Nntp_posting_host    2601
Message_id           6000
Reply_to              923
References           3758
dtype: int64

Saving to csv

In [ ]:
train_x.to_csv('train_x.csv')
test_x.to_csv('test_x.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

## Emails
Extract email address from From and Sender column. From and Sender names were not considered as not much info can be gotten. 
Flag whether the document is from an educational institution by checking the domain for .edu
Extra: extract domain, country etc

In [ ]:
def process_emails(original):
    # Extracting the email addresses from From column
    original['Email_from'] = original.From.map(lambda x : re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) 
                                               if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None 
                                               else np.NaN)

    # Extracting the email addresses from Sender column
    original['Email_sender'] = original.Sender.map(lambda x : np.NaN 
                                                   if pd.isnull(x) 
                                                   else re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) 
                                                   if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None 
                                                   else np.NaN)

    # flag as educational organisation if the email address has .edu domain
    original['Edu_org'] = original.Email_from.map(lambda x : 1 if isinstance(x, basestring) and x[len(x) - 3:].find('edu') is not -1 
                                                  else 0)
    return original

In [ ]:
train_x = process_emails(train_x)
train_x.head()

## Subject
For Subject, remove Re: (maybe indicate that its a reply message), throw to content

In [ ]:
def process_subject(original):
    # Process the subject to remove Re:
    original['Subject_processed'] = original.Subject.map(lambda x : x[x.find("FAQ:") + 5:] 
                                                         if x.find("FAQ:") is not -1 
                                                         else x[x.find("Re:") + 4:] if x.find("Re:") is not -1 and x.find("Re", 5) is -1
                                                         else x[x.find("Re:") + 4:x.find("Re", 5)] if x.find("Re:") is not -1
                                                         else x)

    # Reply_msg = 1 if the document is a reply message, otherwise 0
    original['Reply_msg'] = original.Subject.map(lambda x : 1 
                                                 if x.find("Re:") == 1 
                                                 else 0)

    # Adds the processed subject to Content column
    original['Content'] = original.Content + " " + original.Subject_processed

    # Removes Subject_processed column as they are appended to content column
    del original['Subject_processed']
    return original

In [ ]:
train_x = process_subject(train_x)
train_x.head()

## Date
For Date, extract month and year (year seems to be all in 1993)

In [ ]:
def process_date(original):
    # Extracting the Month from the Date column
    month_dict = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    original['Month'] = original.Date.map(lambda y : ''.join(filter(lambda x: x.lower() in month_dict, y.split())))

    # Extracting the Year from the Date column
    original['Year'] = original.Date.map(lambda x : ''.join([x[x.lower().find(i):].split(' ')[1] 
                                                             for i in month_dict 
                                                             if x.lower().find(i) is not -1]))
    original['Year'] = original.Year.map(lambda x : x[2:] 
                                         if len(x) is 4 
                                         else x)
    return original

In [ ]:
train_x = process_date(train_x)
train_x.head()

Checking the columns of original dataframe

In [ ]:
list(train_x)

The following shows that some columns have missing values, such as Organisation, Sender, Lines, Email_from, and Email_sender (might need to remove email_from and email_sender and subject)

In [ ]:
train_x.count()

Sanity check, when using full dataset, if all years are in 93, remove the column

In [ ]:
train_x.groupby('Year').count()['Content']

After checking, all years (except two cases) are in 1993 so year column is removed.

In [ ]:
del train_x['Year']
print list(train_x)

Checking the Month Column, we see that there is one entry which is empty. Further checks shows that it is because the month is spelt as April instead of the usual Apr, therefore the discrepancy.

In [ ]:
train_x.groupby('Month').count()['Content']

In [ ]:
emp = ''
for i in range(len(train_x.Month)):
    if train_x.Month[i] == '':
        emp = i
print train_x.Date[emp]
train_x.at[emp, 'Month'] = 'Apr'
train_x.groupby('Month').count()['Content']

## Educational Organisation

In [ ]:
print train_x.groupby('Edu_org').count()['Content']
ax = seaborn.countplot(x='Edu_org', data=train_x)
ax.set(title = 'Count of Educational Organisations', xlabel = 'Educational Organisation', xticklabels = ["Yes", 'No'])
plt.show()

In [ ]:
print train_x.groupby('Reply_msg').count()['Content']
ax = seaborn.countplot(x='Reply_msg', data=train_x)
ax.set(title = 'Count of Documents that are replies', xlabel = 'Reply Documents', xticklabels = ["Yes", 'No'])
plt.show()

In [ ]:
train_x.groupby('Organization').count()['Content']

## Spacy Tutorial

In [ ]:
nlp = spacy.load('en')

In [9]:
def clean_text(document):
    # Removes emails
    document = re.sub(r'[\w\.-]+@[\w\.-]+', ' ', document)
    
    # Removes URLs
    document = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', document)
    
    # Removes punctuations
    document = document.translate(None, string.punctuation)
    
    # Change to lowercase
    document = document.lower()
    
    # Removes newline characters
    document = ' '.join(document.split('\n'))
    
    # Removes ASCII
    document = re.sub(r'[^\x00-\x7F]+',' ', document)
    
    # Removes numbers
    document = document.translate(None, digits)
    
    # Replace multiple spaces with one space
    document = ' '.join(document.split())
    #document = nlp(unicode(document))
    
    return document

In [ ]:
def spacy_tokenizer(document):
    parser = English()
    document = parser(unicode(document))
    # Removes stopwords
    document = [tok for tok in document if (str(tok) not in stopwords)]
    return document

In [67]:
train_x_content = train_x.Content.map(lambda x : clean_text(x))
train_x_content = train_x_content.map(lambda x : spacy_tokenizer(x))
test_x_content = test_x.Content.map(lambda x : clean_text(x))
test_x_content = test_x_content.map(lambda x : spacy_tokenizer(x))

## Vectorizing
Transforming into DTM

In [ ]:
count_vect = CountVectorizer()
linear_svc = LinearSVC()
pipe = Pipeline([('count_vect', count_vect), ('linear_svc', linear_svc)])

## Stemming

In [81]:
porter_stemmer = PorterStemmer()
train_x_stem = train_x_content.map(lambda x : ' '.join([porter_stemmer.stem(str(i)) for i in x]))
test_x_stem = test_x_content.map(lambda x : ' '.join([porter_stemmer.stem(str(i)) for i in x]))

In [ ]:
pipe.fit([x for x in train_x_stem], [y for y in y_train])
pred = pipe.predict(test_x_stem)
print "Accuracy:", accuracy_score(y_test, pred)

## Lemmatizing

In [87]:
wordnet_lemmatizer = WordNetLemmatizer()
train_x_lem = train_x_content.map(lambda x : ' '.join([wordnet_lemmatizer.lemmatize(str(i)) for i in x]))
test_x_lem = test_x_content.map(lambda x : ' '.join([wordnet_lemmatizer.lemmatize(str(i)) for i in x]))

In [89]:
pipe.fit([x for x in train_x_lem], [y for y in y_train])
pred = pipe.predict(test_x_lem)
print "Accuracy:", accuracy_score(y_test, pred)

Accuracy: 0.7805


In [ ]:
0.780666666667